In [1]:
pip install meteostat

Note: you may need to restart the kernel to use updated packages.


In [2]:
from datetime import datetime
import matplotlib.pyplot as plt
from meteostat import Point, Daily
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time
import folium
from folium import Marker
from folium.plugins import HeatMap
import matplotlib.pyplot as plt

In [3]:
# Read CSV file
uni_data = pd.read_csv('RMP_Data_Clean_Organized.csv')
uni_data.head()

,University,Safety,Facilities,Social,Average Sentiment
0,Princeton University,4.5,4.4,4.1,0.534225
1,Massachusetts Institute of Technology,4.5,3.7,3.7,0.585213
2,Harvard University,3.2,3.2,2.9,0.249510
3,Stanford University,4.5,4.5,4.0,0.581759
4,Yale University,3.6,4.1,4.0,0.597000


In [4]:
# List of Top 50 Universities

universities = ["Princeton University", "Massachusetts Institute of Technology", "Harvard University",
    "Stanford University", "Yale University", "California Institute of Technology",
    "Duke University", "Johns Hopkins University", "Northwestern University",
    "University of Pennsylvania", "Cornell University", "University of Chicago",
    "Brown University", "Columbia University", "Dartmouth College",
    "University of California-Los Angeles", "University of California-Berkeley", "Rice University",
    "University of Notre Dame", "Vanderbilt University", "Carnegie Mellon University",
    "University of Michigan-Ann Arbor", "Washington University in St. Louis", "Emory University",
    "Georgetown University", "University of Virginia", "University of North Carolina-Chapel Hill",
    "University of Southern California", "University of California-San Diego", "New York University",
    "University of Florida", "University of Texas-Austin", "Georgia Institute of Technology",
    "University of California-Davis", "University of California-Irvine", "University of Illinois-Urbana-Champaign",
    "Boston College", "Tufts University", "University of California-Santa Barbara",
    "University of Wisconsin-Madison", "Boston University", "Ohio State University-Columbus",
    "Rutgers University-New Brunswick", "University of Maryland-College Park", "University of Rochester",
    "Lehigh University", "Purdue University", "University of Georgia", "University of Washington",
    "Wake Forest University"]

In [5]:
# Building a function to find the coordinates of a given location (latitude & longitude)

In [6]:
def find_coordinates(location_name):
    """
    Finds the latitude and longitude of the location (str)
    """
    geolocator = Nominatim(user_agent="geo_locator")
    try:
        location = geolocator.geocode(location_name)
        return location.latitude, location.longitude
    except GeocoderTimedOut:
        time.sleep(1)  
        return find_coordinates(location_name)

In [7]:
# Apply the function to find coordinates and add new columns
uni_data[['Latitude', 'Longitude']] = uni_data['University'].apply(lambda name: pd.Series(find_coordinates(name)))

In [8]:
uni_data.head()

,University,Safety,Facilities,Social,Average Sentiment,Latitude,Longitude
0,Princeton University,4.5,4.4,4.1,0.534225,40.338675,-74.658365
1,Massachusetts Institute of Technology,4.5,3.7,3.7,0.585213,42.358253,-71.096627
2,Harvard University,3.2,3.2,2.9,0.249510,42.365743,-71.122214
3,Stanford University,4.5,4.5,4.0,0.581759,37.431314,-122.169365
4,Yale University,3.6,4.1,4.0,0.597000,41.257131,-72.989670


In [9]:
# Create a map centered on North America
map_na = folium.Map(location=[39.8283, -98.5795], tiles='cartodbpositron', zoom_start=4)

# Add universities to the map based on location
for idx, row in uni_data.iterrows():
    folium.Marker(location=[row["Latitude"], row["Longitude"]],popup=f"{row['University']}").add_to(map_na)

# Display Map
map_na

In [10]:
# Define a function to create a heatmap
def create_heatmap(dataframe, column):
    
    # Initialize a map of North America
    heatmap = folium.Map(location=[39.8283, -98.5795], tiles='cartodbpositron', zoom_start=4)
    
    # Viridis color scale
    viridis_gradient = {
        0.0: '#440154',  # Dark purple
        0.2: '#3b528b',  # Blue
        0.4: '#21908d',  # Teal
        0.6: '#5ec962',  # Green
        0.8: '#fde725',  # Yellow
        1.0: '#ffffbf'   # Light yellow
    }
    
    # Extract Data & Create Layer
    heatmap_data = list(zip(dataframe["Latitude"], dataframe["Longitude"], dataframe[column]))
    HeatMap(data=heatmap_data, radius=20, gradient=viridis_gradient).add_to(heatmap)
    
    return heatmap

In [11]:
# Create heatmaps
heatmap_safety = create_heatmap(uni_data, "Safety")
heatmap_facilities = create_heatmap(uni_data, "Facilities")
heatmap_social = create_heatmap(uni_data, "Social")
heatmap_sentiment = create_heatmap(uni_data, "Average Sentiment")

In [12]:
# Display heatmap for safety score per school
heatmap_safety

In [13]:
# Display heatmap for facilities score per school
heatmap_facilities

In [14]:
# Display heatmap for social score per school
heatmap_social

In [15]:
# Display heatmap for average sentiment score per school
heatmap_sentiment